# KNN with Parameter Tuning

In this project, I implement from scratch the k-NN algorithm on a dataset containing breast cancer data. 
I also implement my own cross-validation algorithm in order to tune the model.

In [108]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math 
import statistics

In [109]:
def euclidean_distance(p, q):
    sum_cathetus = 0
    for i in range(0, len(p)):
        cathetus = (p[i] - q[i])**2
        sum_cathetus += cathetus
    
    return math.sqrt(sum_cathetus)

    
def knn(x_train_normalized, y_train, x_test_normalized, k, distance_func):
    predictions = []
    for row, query_point in x_test_normalized.iterrows():
        distances = []
        labels = []

        #Get distances between each x_train point and the query point
        for index in range(0, len(x_train_normalized)):
            d = distance_func(x_train_normalized.iloc[index], query_point)
            distances.append((d,index))
        distances = sorted(distances)

        k_nearest_neighbors = distances[0:k]

        #Getting the labels of y for each neighbor of the query point
        for neighbor in k_nearest_neighbors:
            index = neighbor[1]
            label = y_train['target'][index]
            labels.append(label)

        prediction = statistics.mode(labels)
        predictions.append(prediction)
        y_predictions = pd.DataFrame(data={'predictions':predictions})

    return y_predictions

def assign_evaluation_metric_label(target, prediction):

    if target == 0 and prediction == 0:
        return 'TN'
    if target == 0 and prediction == 1:
        return 'FP'
    if target == 1 and prediction == 0:
        return 'FN'
    if target == 1 and prediction == 1:
        return 'TP'
    
def get_evaluation_metrics(y_test, y_predictions):
    y_test_predictions_concat = pd.concat([y_test, y_predictions], axis=1)
    evaluation_metric_labels = list(y_test_predictions_concat.apply(lambda row: assign_evaluation_metric_label(row['target'], row['predictions']), axis=1))
    tp = evaluation_metric_labels.count('TP')
    fp = evaluation_metric_labels.count('FP')
    fn = evaluation_metric_labels.count('FN')
    tn = evaluation_metric_labels.count('TN')
    recall = tp/(tp+fn)
    precision = tp/(tp+fp) if tp != 0 else 0
    fpr = fp/(fp + tn)
    f1_score = 2*((precision*recall)/(precision+recall)) if precision + recall != 0 else 0
    accuracy = (tp + tn)/y_test_predictions_concat.shape[0]
    conf_matrix = np.array([[tp, fp],
                       [fn, tn]])
    
    return  conf_matrix, recall, precision, fpr, f1_score, accuracy

def cross_validation(x_train_normalized, y_train,  k_values, number_of_folds, distance_func):
    np.random.seed(23)
    fold_assignments = pd.DataFrame(data={'assignments':np.random.randint(1,number_of_folds+1, size=len(x_train_normalized))})
    k_evaluation_metrics_table = pd.DataFrame(columns=[str(k) for k in k_values])
    for i in number_of_folds:
        x_validation_fold = x_train_normalized[fold_assignments['assignments'] == i]
        y_validation_fold = y_train[fold_assignments['assignments'] == i]
        x_train_folds = x_train_normalized[fold_assignments['assignments'] != i]
        y_train_folds = y_train[fold_assignments['assignments'] != i]

        for k in k_values:
            

    




In [110]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

breast_cancer_train = pd.read_csv("breast_cancer_train.csv")
breast_cancer_test = pd.read_csv("breast_cancer_test.csv")

print(f"Label proportions:\n{breast_cancer_train['target'].value_counts(normalize=1)}")

x_train = breast_cancer_train.drop('target', axis=1)
y_train = breast_cancer_train[['target']]

x_test = breast_cancer_test.drop('target', axis=1)
y_test = breast_cancer_test[['target']]

print(x_train.columns)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

x_train_normalized = ((x_train - x_train.min()) / (x_train.max() - x_train.min()) )
x_test_normalized = ((x_test - x_train.min()) / (x_train.max() - x_train.min()) )

Label proportions:
0    0.657143
1    0.342857
Name: target, dtype: float64
Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')
(455, 30) (455, 1) (114, 30) (114, 1)


### Evaluating k = 31 with Euclidean Distance

In [111]:
y_predictions = knn(x_train_normalized, y_train, x_test_normalized, 31, euclidean_distance)

conf_matrix, recall, precision, fpr, f1_score, accuracy = get_evaluation_metrics(y_test, y_predictions)

print(f' Accuracy: {accuracy}\n Recall: {recall}\n Precision: {precision}\n F1-Score: {f1_score}')

 Accuracy: 0.9385964912280702
 Recall: 0.875
 Precision: 1.0
 F1-Score: 0.9333333333333333
